A good source from... someone at Yale?

https://zoo.cs.yale.edu/classes/cs458/lectures/yfinance.html

https://finance.yahoo.com/quote/AAPL/key-statistics/

In [2]:
import yfinance as yf
import pandas as pd
import plotly as py
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import numpy as np
import seaborn as sns
import datetime as dt
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
warnings.filterwarnings("ignore", category=FutureWarning)

### Data Retrieval & Basic Manipulation

In [3]:
def fetch_stock_data(tickers: list, start_date, end_date, interval="60m"):
    """
    Fetch stock data for given tickers and date range.
    
    Args:
        tickers (list): List of stock ticker symbols (e.g., ['AAPL', 'MSFT']).
        start_date (str): Start date (e.g., '2024-10-01').
        end_date (str): End date (e.g., '2024-10-31').
        interval (str): Interval for data download, valid entries: 
            ‘1m’, ‘2m’, ‘5m’, ‘15m’, ‘30m’, ‘60m’, ‘90m’, ‘1d’, ‘5d’, ‘1wk’, ‘1mo’, ‘3mo’
            recommended: 60m, 1d, 1wk, 1mo.
    
    Returns:
        dict: Dictionary with ticker keys and their data as values, and a list of tickers.
    """
    stock_data = {}

    for ticker in tickers:
        # Download data for each ticker
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval, rounding=True)
        data = data.drop(columns=["Close", "Volume"], errors="ignore")
        stock_data[f"{ticker}_data"] = data
        stock_data["tickers"] = tickers
        
    return stock_data

def correct_stock_data(stock_data):
    """
    Correct stock data by removing multi-index levels and renaming index column.
    
    Args:
        stock_data (dict): Dictionary of stock data with ticker keys.
    """
    for ticker_key, dataframe in stock_data.items():
        if ticker_key == "tickers":
            continue
        dataframe = dataframe.droplevel("Ticker", axis=1)
        dataframe.reset_index(inplace=True)
        dataframe.columns.name = "Data Index"
        if "Datetime" in dataframe.columns:
            dataframe["Datetime"] = pd.to_datetime(dataframe["Datetime"]).dt.strftime('%Y-%m-%d %H:%M')
        stock_data[ticker_key] = dataframe

    return stock_data

def format_large_number(value):
    """Helper function to format large numbers for display."""
    if value is None:
        return None
    elif value >= 1e9:
        return f"{value / 1e9:.2f}B"
    elif value >= 1e6:
        return f"{value / 1e6:.2f}M"
    elif value >= 1e3:
        return f"{value / 1e3:.2f}K"
    else:
        return f"{value:.2f}"

In [ ]:
data = fetch_stock_data(['AAPL', 'MSFT'], '2024-10-01', '2024-11-30', interval='1d')
correct_stock_data(data)

In [9]:
data["AAPL_data"].head()

Data Index,Date,Adj Close,High,Low,Open
0,2024-10-01,225.96,229.65,223.74,229.52
1,2024-10-02,226.53,227.37,223.02,225.89
2,2024-10-03,225.42,226.81,223.32,225.14
3,2024-10-04,226.55,228.00,224.13,227.90
4,2024-10-07,221.45,225.69,221.33,224.50


### Core Data Visualizations

In [4]:
def create_individual_graph(stock_data, file_path="individual_graphs.html"):
    """
    Generate line plots for the adjusted closing prices of stocks and save them as an HTML file.

    Args:
        stock_data (dict): Dictionary of stock data with ticker keys.
        file_path (str): Path for saving the HTML file.

    Returns:
        dict: A dictionary containing the figures and the URL of the saved HTML file.
    """
    figures = {}

    # Generate individual figures for each stock
    for ticker, dataframe in stock_data.items():
        if ticker == "tickers":
            continue

        markers = len(dataframe) <= 62
        render_mode = "svg" if len(dataframe) > 999 else "auto"

        fig = px.line(
            dataframe,
            x=dataframe.columns[0],
            y="Adj Close",
            markers=markers,
            render_mode=render_mode,
            title=f"Price Trend for {ticker}",
        )

        # Define rangebreaks, excluding weekends and (optionally) non-trading hours
        rangebreaks = [dict(bounds=["sat", "mon"])]  # Remove weekends
        if dataframe.columns[0] == "Datetime":
            rangebreaks.append(dict(bounds=[16, 9.5], pattern="hour"))  # Exclude non-trading hours

        fig.update_layout(
            xaxis=dict(rangebreaks=rangebreaks),
            xaxis_title="Date/Time",
            yaxis_title="Adjusted Close Price",
        )

        figures[ticker] = fig

    # Handle multiple figures by combining them into a subplot
    if len(figures) > 1:
        combined_fig = make_subplots(
            rows=1,
            cols=len(figures),
            subplot_titles=[f"Price Trend for {ticker}" for ticker in figures.keys()],
        )

        for i, ticker in enumerate(figures.keys()):
            for trace in figures[ticker]["data"]:
                combined_fig.add_trace(trace, row=1, col=i + 1)

        combined_fig.update_layout(
            xaxis_title="Date/Time",
            yaxis_title="Adjusted Close Price",
        )

        # Save combined figure to HTML file
        pio.write_html(combined_fig, file=file_path, auto_open=False)
        return {"figures": figures, "html_url": file_path}

    # Handle single figure case
    first_ticker = next(iter(figures))  # Get the first (and only) ticker
    pio.write_html(figures[first_ticker], file=file_path, auto_open=False)
    return {"figures": figures, "html_url": file_path}

      
def create_news_table(stock_data, file_path="news_table.html"):
    """
    Create a display table of stock news and save it as an HTML file.

    Args:
        stock_data (dict): Dictionary of stock data with ticker keys.
        file_path (str): Path for saving the HTML file.

    Returns:
        dict: A dictionary containing the Plotly table figure and the URL of the saved HTML file.
    """
    all_news = []

    # Collect news
    for ticker in stock_data["tickers"]:
        ticker_item = yf.Ticker(ticker)
        news = ticker_item.get_news()
        news_df = pd.DataFrame(news)

        # Limit the number of articles
        if len(stock_data["tickers"]) > 1:
            news_df = news_df.iloc[:3]
        else:
            news_df = news_df.iloc[:6]

        all_news.append(news_df)

    # Concatenate news into a single DataFrame
    final_news_df = pd.concat(all_news, ignore_index=True)
    final_news_df = final_news_df.drop(columns=["uuid", "type", "thumbnail", "link"], errors="ignore")
    final_news_df["providerPublishTime"] = pd.to_datetime(
        final_news_df["providerPublishTime"], unit="s"
    ).dt.strftime('%Y-%m-%d %H:%M')

    # Create a Plotly Table figure
    fig = go.Figure(
        data=[
            go.Table(
                header=dict(
                    values=list(final_news_df.columns),
                    fill_color="lightgrey",
                    align="left",
                ),
                cells=dict(
                    values=[final_news_df[col].tolist() for col in final_news_df.columns],
                    fill_color="whitesmoke",
                    align="left",
                ),
            )
        ]
    )

    # Save the table as an HTML file
    pio.write_html(fig, file=file_path, auto_open=False)

    # Return the figure and HTML file path
    return {"figure": fig, "html_url": file_path}


def create_finances_table(stock_data, file_path="finances_table.html"):
    """
    Create a table showing financials, dispersed evenly by date into 10 rows, and save it as an HTML file.

    Args:
        stock_data (dict): Dictionary of stock data with ticker keys.
        file_path (str): Path for saving the HTML file.

    Returns:
        dict: A dictionary containing the Plotly table figure and the URL of the saved HTML file.
    """
    table_subsets = {}

    # Process each ticker's DataFrame
    for ticker, dataframe in stock_data.items():
        if ticker == "tickers":
            continue

        # Reduce DataFrame to 10 rows (dispersed evenly)
        if len(dataframe) <= 10:
            table_subsets[ticker] = dataframe
        else:
            step = max(1, len(dataframe) // 9)
            reduced_df = dataframe.iloc[::step].head(9)
            reduced_df = pd.concat([reduced_df, dataframe.iloc[[-1]]])  # Include the last row

            table_subsets[ticker] = reduced_df

    # Horizontally concatenate DataFrames
    merged_table = pd.concat(table_subsets.values(), axis=1, keys=table_subsets.keys())

    # Create a Plotly Table
    fig = go.Figure(
        data=[
            go.Table(
                header=dict(
                    values=list(merged_table.columns),  # Column headers
                    fill_color="lightgrey",
                    align="left",
                ),
                cells=dict(
                    values=[merged_table[col].tolist() for col in merged_table.columns],  # Data for each column
                    fill_color="whitesmoke",
                    align="left",
                ),
            )
        ]
    )

    # Save the table as an HTML file
    pio.write_html(fig, file=file_path, auto_open=False)

    # Return the figure and HTML file path
    return {"figure": fig, "html_url": file_path}


def create_important_table(stock_data, file_path="important_table.html"):
    """
    Create a secondary table displaying 'important' metrics such as beta, PE ratios, and 52-week scores,
    with tickers as columns and metrics as rows. Save the table as an HTML file.

    Args:
        stock_data (dict): Dictionary of stock data with ticker keys.
        file_path (str): Path for saving the HTML file.

    Returns:
        dict: A dictionary containing the Plotly table figure and the URL of the saved HTML file.
    """
    metrics_list = []

    # Loop through tickers and extract data
    for ticker in stock_data["tickers"]:
        ticker_item = yf.Ticker(ticker)
        metrics = {
            "Ticker": ticker,
            "Trailing P/E": round(ticker_item.info.get("trailingPE", float("nan")), 3),
            "Forward P/E": round(ticker_item.info.get("forwardPE", float("nan")), 3),
            "52 Week Low": round(ticker_item.info.get("fiftyTwoWeekLow", float("nan")), 3),
            "52 Week High": round(ticker_item.info.get("fiftyTwoWeekHigh", float("nan")), 3),
            "52 Week Change": round(ticker_item.info.get("52WeekChange", float("nan")), 3),
            "50 Day Average": round(ticker_item.info.get("fiftyDayAverage", float("nan")), 3),
            "200 Day Average": round(ticker_item.info.get("twoHundredDayAverage", float("nan")), 3),
        }
        metrics_list.append(metrics)

    # Create DataFrame and transpose for table structure
    dataframe = pd.DataFrame(metrics_list).set_index("Ticker").T

    # Create Plotly Table
    fig = go.Figure(
        data=[
            go.Table(
                header=dict(
                    values=["Metrics"] + list(dataframe.columns),
                    fill_color="lightgrey",
                    align="left",
                ),
                cells=dict(
                    values=[dataframe.index.tolist()] + [dataframe[col].tolist() for col in dataframe.columns],
                    fill_color="whitesmoke",
                    align="left",
                ),
            )
        ]
    )

    # Save the table as an HTML file
    pio.write_html(fig, file=file_path, auto_open=False)

    # Return the figure and HTML file path
    return {"figure": fig, "html_url": file_path}


def create_additional_table(stock_data, file_path="additional_table.html"):
    """
    Create tertiary table to display other metrics relevant to stock purchasing decisions.
    Save the table as an HTML file.

    Args:
        stock_data (dict): Dictionary of stock data with ticker keys.
        file_path (str): Path for saving the HTML file.

    Returns:
        dict: A dictionary containing the Plotly table figure and the URL of the saved HTML file.
    """
    metrics_list = []

    for ticker in stock_data["tickers"]:
        ticker_item = yf.Ticker(ticker)
        metrics = {
            "Ticker": ticker,
            "Industry": ticker_item.info.get("industry"),
            "Analyst Recommendation": ticker_item.info.get("recommendationKey"),
            "Beta": ticker_item.info.get("beta"),
            "Overall Risk Score": ticker_item.info.get("overallRisk"),
            "Market Cap": ticker_item.info.get("marketCap"),
            "Total Revenue": ticker_item.info.get("totalRevenue"),
            "Net Profit": ticker_item.info.get("netIncomeToCommon"),
            "Profit Margins": ticker_item.info.get("profitMargins"),
        }

        # Additional Calculations
        total_revenue = metrics["Total Revenue"]
        gross_margins = ticker_item.info.get("grossMargins")
        gross_profit = total_revenue * gross_margins if total_revenue and gross_margins else None
        metrics["Gross Profit"] = gross_profit

        # Format metrics for display
        if metrics["Profit Margins"] is not None:
            metrics["Profit Margins"] = f"{metrics['Profit Margins'] * 100:.2f}%"

        metrics["Market Cap"] = format_large_number(metrics["Market Cap"])
        metrics["Total Revenue"] = format_large_number(metrics["Total Revenue"])
        metrics["Net Profit"] = format_large_number(metrics["Net Profit"])
        metrics["Gross Profit"] = format_large_number(metrics["Gross Profit"])

        # Add to metrics list
        metrics_list.append(metrics)

    # Create DataFrame
    dataframe = pd.DataFrame(metrics_list)

    # Create Plotly Table
    fig = go.Figure(
        data=[
            go.Table(
                header=dict(
                    values=list(dataframe.columns),  # Column headers
                    fill_color="lightgrey",
                    align="left",
                ),
                cells=dict(
                    values=[dataframe[col].tolist() for col in dataframe.columns],  # Data for each column
                    fill_color="whitesmoke",
                    align="left",
                ),
            )
        ]
    )

    # Save the table as an HTML file
    pio.write_html(fig, file=file_path, auto_open=False)

    # Return the figure and HTML file path
    return {"figure": fig, "html_url": file_path}




In [ ]:
data = fetch_stock_data(['AAPL', 'MSFT'], '2024-10-01', '2024-11-30', interval='1d')
correct_stock_data(data)

In [121]:
# data = fetch_stock_data(["AAPL"], "2023-10-01", "2024-10-31", interval="1d")
# correct_stock_data(data)
graph = create_individual_graph(data)
graph['figures']['AAPL_data']

In [116]:
news = create_news_table(data)
news['figure']

In [105]:
finances = create_finances_table(data)
finances['figure']

In [110]:
important = create_important_table(data)
important['figure']

In [119]:
additional = create_additional_table(data)
additional['figure']

### Report Generation

In [5]:
def generate_html_report(data, output_file="final_report.html"):
    """
    Generate an HTML report by combining outputs of various functions and write it to a single file.

    Args:
        data (dict): Stock data to be passed to the individual functions.
        output_file (str): The path to save the final HTML report.

    Returns:
        str: The file path of the generated HTML report.
    """
    # Generate HTML outputs for individual sections
    html_inputs = {
        "Stock Price Chart": create_individual_graph(data)["html_url"],
        "News Table": create_news_table(data)["html_url"],
        "Finances Table": create_finances_table(data)["html_url"],
        "Important Metrics Table": create_important_table(data)["html_url"],
        "Additional Metrics Table": create_additional_table(data)["html_url"],
    }

    # Start building the HTML report
    html_report = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Stock Analysis Report</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; }
            h1 { text-align: center; }
            section { margin-bottom: 40px; }
            iframe { width: 100%; height: 600px; border: none; }
        </style>
    </head>
    <body>
        <h1>Stock Analysis Report</h1>
    """

    # Add each HTML file as an iframe
    for idx, (title, html_file) in enumerate(html_inputs.items(), 1):
        html_report += f"""
        <section>
            <h2>{title}</h2>
            <iframe src="{html_file}"></iframe>
        </section>
        """

    # Close the HTML tags
    html_report += """
    </body>
    </html>
    """

    # Write the final HTML report to a file
    with open(output_file, "w") as file:
        file.write(html_report)

    return output_file


In [129]:
report = generate_html_report(data)

### GUI Workflow

In [ ]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import os

def gui_workflow():
    """
    GUI-based workflow to fetch data, process it, and generate an HTML report.
    """
    # Create the root window
    root = tk.Tk()
    root.withdraw()  # Hide the root window

    # Step 1: Get user input through dialogs
    tickers = simpledialog.askstring("Input Tickers", "Enter stock tickers (comma-separated):")
    start_date = simpledialog.askstring("Input Start Date", "Enter start date (YYYY-MM-DD):")
    end_date = simpledialog.askstring("Input End Date", "Enter end date (YYYY-MM-DD):")
    interval = simpledialog.askstring("Input Time Interval", "Enter time interval (30m, 60m, 1d, 1wk, 1mo) ")

    # Step 2: Fetch and process data
    raw_data = fetch_stock_data(tickers.split(","), start_date, end_date)
    data = correct_stock_data(raw_data)

    # Step 3: Generate HTML report
    output_path = "final_report.html"
    report_path = generate_html_report(data, output_file=output_path)

    # Display success message
    messagebox.showinfo("Success", f"HTML report generated successfully: {os.path.abspath(report_path)}")

if __name__ == "__main__":
    gui_workflow()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


: 

In [144]:
ticker.info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p

In [141]:
ticker = yf.Ticker("AAPL")

news = ticker.get_news()
news

[{'uuid': '4205eaa9-f620-3a0b-a81a-0e82c7c9fd0b',
  'title': 'Magnificent Seven Stocks: Nvidia Stock Extends Rally; Amazon, Tesla Hit New Highs',
  'publisher': "Investor's Business Daily",
  'link': 'https://finance.yahoo.com/m/4205eaa9-f620-3a0b-a81a-0e82c7c9fd0b/magnificent-seven-stocks%3A.html',
  'providerPublishTime': 1733420922,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/ioB_fyxV7Zn4zyY.eCcinA--~B/aD01NjM7dz0xMDAwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ibd.com/ad2ae2e036e85be47f2c44af5d34923d',
     'width': 1000,
     'height': 563,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/wsc.KBRTJJlBxNK8WieAEQ--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ibd.com/ad2ae2e036e85be47f2c44af5d34923d',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['NVDA', 'AAPL', 'TSLA', 'META', 'MSFT']},
 {'uuid': '5ab5e48a-baa5-3c5b-af5a-548d2

In [ ]:
dir(ticker)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_analysis',
 '_data',
 '_download_options',
 '_earnings',
 '_earnings_dates',
 '_expirations',
 '_fast_info',
 '_fetch_ticker_tz',
 '_financials',
 '_fundamentals',
 '_funds_data',
 '_get_ticker_tz',
 '_holders',
 '_isin',
 '_lazy_load_price_history',
 '_news',
 '_options2df',
 '_price_history',
 '_quote',
 '_shares',
 '_tz',
 '_underlying',
 'actions',
 'analyst_price_targets',
 'balance_sheet',
 'balancesheet',
 'basic_info',
 'calendar',
 'capital_gains',
 'cash_flow',
 'cashflow',
 'dividends',
 'earnings',
 'earnings_dates',
 'earnings_estimate',
 'earnings_history',
 'eps_revisions',
 'eps_trend',


In [142]:
for key, value in ticker.info.items():
    print(f"{key}: {value}")

address1: One Apple Park Way
city: Cupertino
state: CA
zip: 95014
country: United States
phone: (408) 996-1010
website: https://www.apple.com
industry: Consumer Electronics
industryKey: consumer-electronics
industryDisp: Consumer Electronics
sector: Technology
sectorKey: technology
sectorDisp: Technology
longBusinessSummary: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangement

In [ ]:
# PE ratio will look something like this
ticker.info["previousClose"] / (ticker.info["netIncomeToCommon"] / ticker.info["sharesOutstanding"])

39.12956282740667

In [ ]:
ticker.info["dividendYield"]

0.0041

In [ ]:
ticker.info["beta"]

1.24

In [ ]:
ticker = yf.Ticker("AAPL")
history = ticker.history(period="5y")
history

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-12-05 00:00:00-05:00,63.934952,64.443931,63.678039,64.368790,74424400,0.0,0.0
2019-12-06 00:00:00-05:00,64.829292,65.682434,64.785660,65.612144,106075600,0.0,0.0
2019-12-09 00:00:00-05:00,65.440071,65.633965,64.206406,64.693573,128042400,0.0,0.0
2019-12-10 00:00:00-05:00,65.100761,65.457047,64.436661,65.071678,90420400,0.0,0.0
2019-12-11 00:00:00-05:00,65.151634,65.706665,65.076500,65.626678,78756800,0.0,0.0
...,...,...,...,...,...,...,...
2024-11-27 00:00:00-05:00,234.470001,235.690002,233.809998,234.929993,33498400,0.0,0.0
2024-11-29 00:00:00-05:00,234.809998,237.809998,233.970001,237.330002,28481400,0.0,0.0
2024-12-02 00:00:00-05:00,237.270004,240.789993,237.160004,239.589996,48137100,0.0,0.0


In [ ]:
ticker.major_holders

Breakdown,Value
insidersPercentHeld,0.02056
institutionsPercentHeld,0.61903
institutionsFloatPercentHeld,0.63203
institutionsCount,6485.00000


In [ ]:
ticker.institutional_holders

,Date Reported,Holder,pctHeld,Shares,Value
0,2023-06-30,Vanguard Group Inc,0.0834,1303688506,252876459508
1,2023-06-30,Blackrock Inc.,0.0665,1039640859,201659137420
2,2023-06-30,"Berkshire Hathaway, Inc",0.0586,915560382,177591247296
3,2023-06-30,State Street Corporation,0.0370,578897858,112288817516
4,2023-06-30,"FMR, LLC",0.0196,307066638,59561715772
5,2023-06-30,"Geode Capital Management, LLC",0.0186,291538165,56549657865
6,2023-06-30,Price (T.Rowe) Associates Inc,0.0145,226650943,43963483413
7,2023-06-30,Morgan Stanley,0.0131,204714950,39708558851
8,2022-12-31,Norges Bank Investment Management,0.0107,167374278,21746939940
9,2023-06-30,Northern Trust Corporation,0.0105,164536073,31915062079


In [ ]:
ticker.calendar

{'Dividend Date': datetime.date(2024, 11, 13),
 'Ex-Dividend Date': datetime.date(2024, 11, 7),
 'Earnings Date': [datetime.date(2025, 1, 30), datetime.date(2025, 2, 3)],
 'Earnings High': 2.5,
 'Earnings Low': 2.19,
 'Earnings Average': 2.35487,
 'Revenue High': 129887000000,
 'Revenue Low': 119563000000,
 'Revenue Average': 124365081850}

In [ ]:
ticker.recommendations

,period,strongBuy,buy,hold,sell,strongSell
0,0m,8,24,12,1,2
1,-1m,8,24,12,1,2
2,-2m,8,23,12,1,2
3,-3m,8,24,12,0,2


In [ ]:
ticker.sustainability

,esgScores
maxAge,86400
totalEsg,16.79
environmentScore,0.55
socialScore,7.52
governanceScore,8.72
ratingYear,2024
ratingMonth,11
highestControversy,3.0
peerCount,238
esgPerformance,LAG_PERF
